# Introduction
train and test data are measured with same roll resistance (lever position 3) and wheel pressure.


# Analyze linearity of train vs test data
Question: Is the velocity a good feature to predict the virtual power? To find out we create a linear/polynomial model or KNN regression model from the app data and calculate a score for each tacx cadence value to see whether the model predicts the velocity good enough.
First let us see (plot) and calculate (relation of cadence/velocity is constant per gear)  the training data.

_Train data (app data):
* X of app data set -> cadence
* Y -> velocity

## Pre processing


In [271]:
import pandas as pd
import numpy as np
from src.tcx import read_tcx

tcx: dict = read_tcx('test/tcx/cadence_1612535177298-gear7.tcx')


trackpoints: dict = tcx['TrainingCenterDatabase']['Activities']['Activity']['Lap']['Track']
trackpoint_dict_list = list(trackpoints.values())[0]

tcx_df: pd.DataFrame = pd.DataFrame.from_records(trackpoint_dict_list)

tcx_df['DistanceMeters'] = tcx_df['DistanceMeters'].apply(lambda x: float(x))
tcx_df['DistanceMeters']

# print(tcx_df['Extensions'])
tcx_df['Ext.Watts'] = [extension_dict['TPX']['Watts'] for extension_dict in tcx_df['Extensions']]
tcx_df['Ext.Speed'] = [extension_dict['TPX']['Speed'] for extension_dict in tcx_df['Extensions']]


# Distance delta
tcx_df['DistanceMeters-delta'] = pd.Series(
    data=np.subtract(tcx_df['DistanceMeters'].iloc[1:].to_numpy(),tcx_df['DistanceMeters'].iloc[0:-1].to_numpy()),
    index=range(1, len(tcx_df)))

# Time delta
## Time -> pd.Timestamp
tcx_df['Time'] = tcx_df['Time'].apply(lambda t: pd.to_datetime(t))
## Time[i+1] - Time[i] type = pd.Timedelta
tcx_df['Time-delta'] = pd.Series(
    data=np.subtract(tcx_df['Time'].iloc[1:].to_numpy(),tcx_df['Time'].iloc[0:-1].to_numpy()),
    index=range(1, len(tcx_df)))

## Speed [km/h] = distance [meter] / time-delta [second] * 3.6
tcx_df['Speed'] = tcx_df['DistanceMeters-delta'] / tcx_df['Time-delta'].apply(lambda td: td.total_seconds()) * 3.6

## speed / cadence
tcx_df['Speed-per-cadence'] = tcx_df['Speed'] / tcx_df['Cadence'].apply(lambda x: float(x))

tcx_df[['Speed', 'Cadence', 'Ext.Watts', 'DistanceMeters', 'DistanceMeters-delta', 'Ext.Speed', 'Time', 'Time-delta']].head(20)

tcx_df.describe()

# df = pd.json_normalize([{'activity': {'id': 'tacx 12.11.'}}, {'activity': {'id': 'olli'}}]);
# df

,DistanceMeters,DistanceMeters-delta,Time-delta,Speed,Speed-per-cadence
count,188.000000,187.000000,187,187.000000,187.000000
mean,630.537784,7.165124,0 days 00:00:01.041647058,24.765209,0.275846
std,392.070789,0.896864,0 days 00:00:00.006815031,3.113125,0.019499
min,5.312400,5.102700,0 days 00:00:01.007000,17.663192,0.265494
25%,281.129063,6.640500,0 days 00:00:01.040000,22.969182,0.273076
50%,611.275500,7.304550,0 days 00:00:01.041000,25.260692,0.274223
75%,961.727888,7.793850,0 days 00:00:01.044000,26.965729,0.275529
max,1345.190550,8.737500,0 days 00:00:01.073000,30.303468,0.536816


## Linear regression